In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_excel("BlinkIT Grocery Data.xlsx", sheet_name="BlinkIT Grocery Data")
# Display basic info
print("Initial dataset info:")
print(df.info())

Initial dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item Fat Content           8523 non-null   object 
 1   Item Identifier            8523 non-null   object 
 2   Item Type                  8523 non-null   object 
 3   Outlet Establishment Year  8523 non-null   int64  
 4   Outlet Identifier          8523 non-null   object 
 5   Outlet Location Type       8523 non-null   object 
 6   Outlet Size                8523 non-null   object 
 7   Outlet Type                8523 non-null   object 
 8   Item Visibility            8523 non-null   float64
 9   Item Weight                7060 non-null   float64
 10  Sales                      8523 non-null   float64
 11  Rating                     8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB
None


In [3]:
import pandas as pd

# Load and convert
df = pd.read_excel("BlinkIT Grocery Data.xlsx", sheet_name="BlinkIT Grocery Data")
df.to_csv("blinkit_data.csv", index=False)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg

# Initialize Spark session
spark = SparkSession.builder \
    .appName("BlinkitDataCleaning") \
    .getOrCreate()

# Load CSV
df_spark = spark.read.option("header", "true").option("inferSchema", "true").csv("blinkit_data.csv")

# Fill missing Item Weight with mean
mean_weight = df_spark.select(avg("Item Weight")).collect()[0][0]
df_spark = df_spark.fillna({"Item Weight": mean_weight})

# Show preview
df_spark.show(5)


+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+----------+
|Item Fat Content|Item Identifier|           Item Type|Outlet Establishment Year|Outlet Identifier|Outlet Location Type|Outlet Size|      Outlet Type|Item Visibility|Item Weight|   Sales|Rating|Outlet Age|
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+----------+
|         Regular|          FDX32|Fruits and Vegeta...|                     2012|           OUT049|              Tier 1|     Medium|Supermarket Type1|      0.1000135|       15.1|145.4786|   5.0|        13|
|         Low Fat|          NCB42|  Health and Hygiene|                     2022|           OUT018|              Tier 3|     Medium|Supermarket Type2|    0.008596051|       11.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg

# STEP 1: Create SparkSession with MongoDB Connector
spark = SparkSession.builder \
    .appName("BlinkitDataToMongo") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.2.0") \
    .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1/BlinkitSalesDB.sales_data") \
    .getOrCreate()

# STEP 2: Load the CSV File
df = spark.read.option("header", "true").option("inferSchema", "true").csv("blinkit_data.csv")

# STEP 3: Clean 'Item Fat Content'
df = df.withColumn("Item Fat Content", 
    when(col("Item Fat Content").isin("LF", "low fat"), "Low Fat")
    .when(col("Item Fat Content") == "reg", "Regular")
    .otherwise(col("Item Fat Content"))
)

# STEP 4: Fill Missing 'Item Weight'
mean_weight = df.select(avg("Item Weight")).first()[0]
df = df.fillna({"Item Weight": mean_weight})

# STEP 5: Create 'Outlet Age'
df = df.withColumn("Outlet Age", 2025 - col("Outlet Establishment Year"))

# Optional: Preview
df.show(5)

# STEP 6: Write to MongoDB
df.write.format("mongodb").mode("append").save()

print("✅ Data successfully written to MongoDB!")

# Stop Spark session
spark.stop()

+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+----------+
|Item Fat Content|Item Identifier|           Item Type|Outlet Establishment Year|Outlet Identifier|Outlet Location Type|Outlet Size|      Outlet Type|Item Visibility|Item Weight|   Sales|Rating|Outlet Age|
+----------------+---------------+--------------------+-------------------------+-----------------+--------------------+-----------+-----------------+---------------+-----------+--------+------+----------+
|         Regular|          FDX32|Fruits and Vegeta...|                     2012|           OUT049|              Tier 1|     Medium|Supermarket Type1|      0.1000135|       15.1|145.4786|   5.0|        13|
|         Low Fat|          NCB42|  Health and Hygiene|                     2022|           OUT018|              Tier 3|     Medium|Supermarket Type2|    0.008596051|       11.

Py4JJavaError: An error occurred while calling o67.save.
: java.lang.BootstrapMethodError: java.lang.NoClassDefFoundError: com/mongodb/ConnectionString
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.addConnectionStringDatabaseAndCollection(AbstractMongoConfig.java:312)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.createUsageOptions(AbstractMongoConfig.java:290)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.lambda$new$1(AbstractMongoConfig.java:82)
	at scala.Option.map(Option.scala:230)
	at com.mongodb.spark.sql.connector.config.AbstractMongoConfig.<init>(AbstractMongoConfig.java:82)
	at com.mongodb.spark.sql.connector.config.WriteConfig.<init>(WriteConfig.java:242)
	at com.mongodb.spark.sql.connector.config.MongoConfig.writeConfig(MongoConfig.java:79)
	at com.mongodb.spark.sql.connector.config.MongoConfig.toWriteConfig(MongoConfig.java:239)
	at com.mongodb.spark.sql.connector.MongoTable.newWriteBuilder(MongoTable.java:111)
	a